## Analyse de la variance des SoH

Ce notebook a pour objectif d'analyser la variance des SoH calculé avant filtrage.


In [ ]:
# from transform.raw_results.tesla_results import get_results 
from core.s3.s3_utils import S3Service, S3Settings
import plotly.express as px
from core.spark_utils import create_spark_session
import os

# Data download & preproc

In [ ]:
spark_session = create_spark_session(S3Settings().S3_KEY,
                             S3Settings().S3_SECRET,)

In [ ]:
df = S3Service().read_parquet_df_spark(spark_session, 'raw_results/tesla.parquet')

In [ ]:
# on drop les valeur vraiement abhérantes
df_filtered = df[
    (df['soh'] >= .5) & (df['soh'] <= 1.5)
]

df_filtered = df_filtered.toPandas()

In [ ]:
# calcul de la variance du SoH et du nombre de fois qu'on l'a calculé
grouped = df_filtered.groupby('vin', as_index=False, observed=True)['soh'].agg(['mean', 'var', 'count'])
grouped.rename(columns={'var': 'soh_variance'}, inplace=True)

In [ ]:
grouped.sort_values('count').dropna()

# Graph & Analysis

In [ ]:
px.histogram(grouped, x="soh_variance",)

On à une variance qui est inférieur à 0.005 pour la majorité de nos SoH

In [ ]:
px.scatter(df_filtered, y='soh', x='odometer' ,color='vin')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


sample_sizes = [2, 5, 10, 20, 30, 50, 70]  # nombre de mesures par véhicule
min_required_measurements = max(sample_sizes)

# Garder seulement les véhicules avec assez de données
vehicles_with_enough_data = df_filtered['vin'].value_counts()
valid_vehicles = vehicles_with_enough_data[vehicles_with_enough_data >= min_required_measurements].index

df_valid = df_filtered[df_filtered['vin'].isin(valid_vehicles)]

# Stocker les résultats ici
results = []

# Boucle sur différentes tailles d'échantillons
for n in sample_sizes:
    variance_list = []

    for veh_id in valid_vehicles:
        sub = df_valid[df_valid['vin'] == veh_id].sort_values('odometer')
        sample = sub.head(n)  # tu peux aussi faire un random sample : sample = sub.sample(n)
        var_soh = sample['soh'].var()
        if not np.isnan(var_soh):
            variance_list.append(var_soh)
    
    avg_var = np.mean(variance_list)
    std_var = np.std(variance_list)
    results.append((n, avg_var, std_var))

# Convertir en DataFrame pour tracer
df_results = pd.DataFrame(results, columns=['sample_size', 'avg_variance', 'std_variance'])

# Tracé
plt.figure(figsize=(10, 5))
plt.plot(df_results['sample_size'], df_results['avg_variance'], marker='o', label='Variance moyenne')
plt.fill_between(df_results['sample_size'],
                 df_results['avg_variance'] - df_results['std_variance'],
                 df_results['avg_variance'] + df_results['std_variance'],
                 alpha=0.2, label='±1 écart-type')
plt.xlabel("Nombre de mesures par véhicule")
plt.ylabel("Variance moyenne du SoH")
plt.title("Convergence de la variance du SoH selon la taille d'échantillon")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

On voit que la variance augmente légèrement. Mais surtout l'interval de l'écart type est très grand c'est du au fait qu'on laisse beaucoup de SoH valide entre .5 et 1.5

Si on décide de filtrer un peu plus la donnée:

In [ ]:
# on drop les valeur vraiement abhérantes
df_filtered = df[
    (df['soh'] >= .6) & (df['soh'] <= 1.2)
].toPandas()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


sample_sizes = [2, 5, 10, 20, 30, 50, 70]  # nombre de mesures par véhicule
min_required_measurements = max(sample_sizes)

# Garder seulement les véhicules avec assez de données
vehicles_with_enough_data = df_filtered['vin'].value_counts()
valid_vehicles = vehicles_with_enough_data[vehicles_with_enough_data >= min_required_measurements].index

df_valid = df_filtered[df_filtered['vin'].isin(valid_vehicles)]

# Stocker les résultats ici
results = []

# Boucle sur différentes tailles d'échantillons
for n in sample_sizes:
    variance_list = []

    for veh_id in valid_vehicles:
        sub = df_valid[df_valid['vin'] == veh_id].sort_values('odometer')
        sample = sub.head(n)  # tu peux aussi faire un random sample : sample = sub.sample(n)
        var_soh = sample['soh'].var()
        if not np.isnan(var_soh):
            variance_list.append(var_soh)
    
    avg_var = np.mean(variance_list)
    std_var = np.std(variance_list)
    results.append((n, avg_var, std_var))

# Convertir en DataFrame pour tracer
df_results = pd.DataFrame(results, columns=['sample_size', 'avg_variance', 'std_variance'])

# Tracé
plt.figure(figsize=(10, 5))
plt.plot(df_results['sample_size'], df_results['avg_variance'], marker='o', label='Variance moyenne')
plt.fill_between(df_results['sample_size'],
                 df_results['avg_variance'] - df_results['std_variance'],
                 df_results['avg_variance'] + df_results['std_variance'],
                 alpha=0.2, label='±1 écart-type')
plt.xlabel("Nombre de mesures par véhicule")
plt.ylabel("Variance moyenne du SoH")
plt.title("Convergence de la variance du SoH selon la taille d'échantillon")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Écart-type élevé pour les petits échantillons
La bande bleue (±1 std) est très large entre 2 et 10 mesures, signe d’une grande incertitude.
Cela s’explique car :
- Peu de données = variance moins fiable

Certains véhicules peuvent avoir des valeurs extrêmes.
Mais la variance moyenne du SoH est relativement faible et stable, autour de 0.004–0.005.
Cela indique que, même avec peu de mesures, la variance reste globalement maîtrisée.

### Coefficient de variation

In [ ]:
def compute_cv(df):
    stats = df.groupby('vin', as_index=False, observed=True)['soh'].agg(['mean', 'std', 'count'])
    stats['cv_percent'] = (stats['std'] / stats['mean']) * 100
    return stats

In [ ]:
cv_stats = compute_cv(df_filtered)

In [ ]:
seuil_cv = 5
vehicules_instables = cv_stats[cv_stats['cv_percent'] > seuil_cv]

# 4. Affichage
print(f"Nombre total de véhicules : {len(cv_stats)}")
print(f"Véhicules avec CV > {seuil_cv}% : {len(vehicules_instables)}")
print(f"Proportion de véhicules instables :  {round(len(vehicules_instables) / len(cv_stats) *100)}%")

In [ ]:
cv_stats

In [ ]:
fig = px.scatter(
    cv_stats,
    x="mean",
    y="cv_percent",
    color="cv_percent",
    title="Coefficient de Variation (CV) du SoH par véhicule",
    labels={
        "mean": "SoH",
        "cv_percent": "CV (%)"
    },
    hover_data=["mean", "std"]
)

# Ajoute une ligne horizontale pour le seuil
fig.add_shape(
    type="line",
    x0=cv_stats['mean'].min(),
    x1=cv_stats['mean'].max(),
    y0=5,
    y1=5,
    line=dict(color="red", dash="dash"),
)
